Taking a look at the generation of the cloud mask, and in particular the "dark pixels" generated by the scene classification layer

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio